In [1]:
from gurobipy import Model, quicksum, GRB

In [2]:
Cust = set(range(1,3))
DC = Cust.copy() 
Route = {(i,j) for i in DC for j in Cust}
CT = {1,2}

T = 3
Period = list(range(1, T+1))

In [3]:
transport = {i:10 for i in DC}
delivery = {(i,j):5 for i,j in Route}

dc_run = {(i,ct):100*ct for i in DC for ct in CT}
dc_stock = {i:1 for i in DC}
dc_ub = {ct:80*ct for ct in CT}
dc_new = 0
dc_cancel = 0

cust_stock = {j:5 for j in Cust}
cust_out = 100

dc_disposal = 10

demand = {(j,t):10 for j in Cust for t in Period}

In [4]:
model = Model()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [5]:
X, x, y, sD, sC, z, w, o, d = {}, {}, {}, {}, {}, {}, {}, {}, {}

for i in DC:
    for t in Period:
        X[i,t] = model.addVar(vtype="I", name=f'X[{i},{t}]')

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="I", name=f'x[{i},{j},{t}]')

for i in DC:
    for ct in CT:
        y[i,ct,0] = model.addVar(vtype="B", ub=0, name=f'y[{i},{ct},{0}]')
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

for i in DC:
    sD[i,0] = model.addVar(vtype="I", ub=0, name=f'sD[{i},{t}]')
    for t in Period:
        sD[i,t] = model.addVar(vtype="I", name=f'sD[{i},{t}]')
        d[i,t] = model.addVar(vtype="I", name=f'd[{i},{t}]')

for j in Cust:
    sC[j,0] = model.addVar(vtype="I", ub=0, name=f'sC[{j},{t}]')
    for t in Period:
        sC[j,t] = model.addVar(vtype="I", name=f'sC[{j},{t}]')
        o[j,t] = model.addVar(vtype="I", name=f'o[{j},{t}]')

for i in DC:
    for ct in CT:
        for t in Period:
            z[i,ct,t] = model.addVar(vtype="B", name=f'z[{i},{ct},{t}]')
            w[i,ct,t] = model.addVar(vtype="B", name=f'w[{i},{ct},{t}]')

model.update()

In [6]:
Cust_Demand_Cons, DC_Running_Cons, CT_Only_Cons, DC_Flow_Cons, DC_UB_Cons, DC_Connect_Cons = {}, {}, {}, {}, {}, {}

for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC) + sC[j,t-1]
            ==
            demand[j,t] + sC[j,t] - o[j,t]
        )

for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t_] for t_ in Period) * quicksum(y[i,ct,t] for ct in CT)
        )

for i in DC:
    for t in Period:
        CT_Only_Cons[i,t] = model.addConstr(
            quicksum(y[i,ct,t] for ct in CT)
            <=
            1
        )

for i in DC:
    for t in Period:
        DC_Flow_Cons = model.addConstr(
            X[i,t] + sD[i,t-1]
            ==
            quicksum(x[i,j,t] for j in Cust) + sD[i,t] + d[i,t]
        )

for i in DC:
    for t in Period:
        DC_UB_Cons[i,t] = model.addConstr(
            X[i,t] + sD[i,t-1]
            <=
            quicksum(dc_ub[ct] * y[i,ct,t] for ct in  CT)
        )

for i in DC:
    for ct in CT:
        for t in Period:
            DC_Connect_Cons[i,ct,t] = model.addConstr(
                y[i,ct,t] - y[i,ct,t-1]
                ==
                z[i,ct,t] - w[i,ct,t]
            )
    
model.update()

In [7]:
model.setObjective(
    quicksum(transport[i] * X[i,t] for i in DC for t in Period) +
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +
    quicksum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(dc_stock[i] * sD[i,t] for i in DC for t in Period) +
    quicksum(cust_stock[j] * sC[j,t] for j in Cust for t in Period) +
    quicksum(dc_new * z[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(dc_cancel * w[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(cust_out * o[j,t] for j in Cust for t in Period) +
    quicksum(dc_disposal * d[i,t] for i in DC for t in Period)
    ,GRB.MINIMIZE
)

model.update()

In [8]:
model.optimize()

Optimize a model with 48 rows, 86 columns and 186 nonzeros
Variable types: 0 continuous, 86 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 6000.0000000
Presolve removed 14 rows and 48 columns
Presolve time: 0.00s
Presolved: 34 rows, 38 columns, 114 nonzeros
Variable types: 0 continuous, 38 integer (12 binary)

Root relaxation: objective 1.066667e+03, 16 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1066.66667    0    2 6000.00000 1066.66667  82.2%     -    0s
H    0     0                    1200.0000000 1066.66667  11.1%     -    0s
     0     0 1100.00000    0    2 1200.00000 1100.00000  8.33%     -    0s
     0     0 1150.00000    0   13 1200.00000 1150.00000  4.17%     -    0

In [9]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

X[1,1] 20.0
X[2,2] 20.0
X[2,3] 20.0
x[1,2,1] 10.0
x[1,1,1] 10.0
x[2,1,2] 10.0
x[2,1,3] 10.0
x[2,2,2] 10.0
x[2,2,3] 10.0
y[1,1,1] 1.0
y[2,1,2] 1.0
y[2,1,3] 1.0
z[1,1,1] 1.0
w[1,1,2] 1.0
z[1,2,1] 1.0
w[1,2,1] 1.0
z[2,1,1] 1.0
w[2,1,1] 1.0
z[2,1,2] 1.0
z[2,2,1] 1.0
w[2,2,1] 1.0
